<a href="https://colab.research.google.com/github/Tannongma/SCM.275x/blob/main/SCM_275x_Transshipment_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Transshipment Problem**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 27.0 MB/s eta 0:00:00


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Plotting nodes on a map**

In [ ]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               color,                       # Color of the marker icon
               background_color,            # Background color of the marker icon
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=color,
                border_color=color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Plotting flows on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows(map,                   # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               max_width = 30,        # Maximum line width for the flows, default is 30
               color = 'grey',        # Color of the lines representing flows, default is grey
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

            # Get the coordinates of the nodes for plotting the line
            points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


            # Add a PolyLine to the map to represent the flow between the nodes
            folium.PolyLine(points,
                            color=color,                                # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map


## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/transshipment_problem/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,country,demand,lat,lon
0,c1,Lyon,France,600,45.7600,4.8400
1,c2,Ludwigshafen,Germany,1000,49.4811,8.4353
2,c3,Pecs,Hungary,700,46.0711,18.2331
3,c4,Dordrecht,Netherlands,500,51.7958,4.6783
4,c5,Haarlem,Netherlands,500,52.3833,4.6333


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/transshipment_problem/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,country,supply,lat,lon
0,s1,Riga,Latvia,3400,56.9489,24.1064
1,s2,Vaciamadrid,Spain,3600,40.3394,-3.5181
2,s3,Almada,Portugal,2000,38.6803,-9.1583
3,s4,Hamburg,Germany,2500,53.6900,10.0357
4,s5,Gdansk,Poland,1500,54.3489,18.5860


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/transshipment_problem/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,country,capacity,lat,lon
0,w1,Clermont-Ferrand,France,1500,45.7831,3.0824
1,w2,Deventer,Netherlands,5000,52.2500,6.1500
2,w3,Gdynia,Poland,2500,54.5175,18.5400
3,w4,Pforzheim,Germany,3000,48.8950,8.7050
4,w5,Budapest,Hungary,1500,47.5151,19.0620


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon, demand):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude
        self.demand = demand      # Customer's demand


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon, supply):
        self.ID = ID            # Supplier's ID
        self.name = name        # Supplier's name
        self.lat = lat          # Supplier's latitude
        self.lon = lon          # Supplier's longitude
        self.supply = supply    # Supplier's available supply


In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, capacity):
        self.ID = ID            # Warehouse's ID
        self.name = name        # Warehouse's name
        self.lat = lat          # Warehouse's latitude
        self.lon = lon          # Warehouse's longitude
        self.capacity = capacity # Warehouse's capacity


#### Creating node objects

In [ ]:
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'],         # Customer's longitude
                                    demand=row['demand'])   # Customer's demand

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'],         # Supplier's longitude
                                    supply=row['supply'])   # Supplier's available supply

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],             # Warehouse's ID
                                    name = row['name'],           # Warehouse's name
                                    lat = row['lat'],             # Warehouse's latitude
                                    lon = row['lon'],             # Warehouse's longitude
                                    capacity = row['capacity'])   # Warehouse's capacity

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([50, 10.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between suppliers and warehouses and warehouses and customers

distances = dict()
for s, supplier in suppliers.items():
  for w, warehouse in warehouses.items():
      distances[s, w] = compute_geodesic_distance(origin = supplier, destination = warehouse, unit = 'km')

for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km_supplier_warehouse = 15  # Cost per unit per kilometer from supplier to warehouse
cost_unit_km_warehouse_customer = 20  # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        unit_cost[s, w] = distances[s, w] * cost_unit_km_supplier_warehouse                    # Calculate unit cost as distance multiplied by cost per km (supplier to warehouse)

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


## **Optimization model**

### **Creating and solving the optimization model**

In [ ]:
# Initializing the model
model = grb.Model("Transshipment Problem")

# Creating decision variables

# Decision variable representing the flows between suppliers and warehouses, and warehouses and customers
flow_vars = dict()
for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        flow_vars[s, w] = model.addVar(vtype=grb.GRB.CONTINUOUS,                               # Flow from supplier to warehouse
                                       name="flow_{0}_{1}".format(s, w))

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        flow_vars[w, c] = model.addVar(vtype=grb.GRB.CONTINUOUS,                               # Flow from warehouse to customer
                                       name="flow_{0}_{1}".format(w, c))


# Calculating total cost for flows between suppliers and warehouses
cost_supplier_warehouse = grb.quicksum(unit_cost[s, w] * flow_vars[s, w]
                                       for s, supplier in suppliers.items()
                                       for w, warehouse in warehouses.items())

# Calculating total cost for flows between warehouses and customers
cost_warehouse_customer = grb.quicksum(unit_cost[w, c] * flow_vars[w, c]
                                       for w, warehouse in warehouses.items()
                                       for c, customer in customers.items())

# Creating the total cost expression
total_cost = cost_supplier_warehouse + cost_warehouse_customer

# Setting the objective function
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints
for c, customer in customers.items():                                                                         # Iterate over customers
    model.addConstr(grb.quicksum(flow_vars[w, c] for w, warehouse in warehouses.items()) == customer.demand)  # Flow into customer must meet demand

# Adding supply constraints
for s, supplier in suppliers.items():                                                                         # Iterate over suppliers
    model.addConstr(grb.quicksum(flow_vars[s, w] for w, warehouse in warehouses.items()) <= supplier.supply)  # Flow out of supplier cannot exceed supply

# Adding flow preservation constraints
for w, warehouse in warehouses.items():                                                                       # Iterate over warehouses
    model.addConstr(grb.quicksum(flow_vars[s, w] for s, supplier in suppliers.items()) ==                     # Flow into warehouse must equal flow out
                    grb.quicksum(flow_vars[w, c] for c, customer in customers.items()))

# Solving the model
model.optimize()


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 44 rows, 259 columns and 518 nonzeros
Model fingerprint: 0x79610d89
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+03]
Presolve time: 0.09s
Presolved: 44 rows, 259 columns, 518 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5681072e+07   2.000000e+04   0.000000e+00      0s
      43    3.2937739e+08   0.000000e+00   0.000000e+00      0s

Solved in 43 iterations and 0.14 seconds (0.00 work units)
Optimal objective  3.293773856e+08


## **Solution visualization and analysis**

In [ ]:
# Visualizing the customer and supplier locations on the map

# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([50, 10.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', color='orange', background_color='yellow')

# Plot the flows between suppliers and customers with a maximum line width of 20
plot_flows(map=map, max_width=20, vars=flow_vars, nodes = nodes)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map



In [ ]:
# ------------------
# OPTIONAL
# ------------------

# This snippet of code shows how to display results in tabular format

# Extracting flow values and creating a DataFrame
flow_data = []
for (origin, destination), var in flow_vars.items():
    if var.X > 0:  # Only include non-zero flows for simplicity
        flow_data.append({'Origin': origin, 'Destination': destination, 'Flow Value': var.X})

# Creating and displaying the DataFrame
flow_df = pd.DataFrame(flow_data)
flow_df

,Origin,Destination,Flow Value
0,s1,w3,2100.0
1,s1,w5,1100.0
2,s1,w6,200.0
3,s2,w1,3600.0
4,s3,w1,2000.0
5,s4,w2,2300.0
6,s4,w6,200.0
7,s5,w6,1500.0
8,s6,w5,2000.0
9,s7,w2,700.0


In [ ]:
# Aggregating data according to origin
aggregated_df = flow_df.groupby('Origin', as_index = False).agg({'Flow Value': 'sum'})


In [ ]:
# Supplier data

supplier_df = aggregated_df.loc[aggregated_df['Origin'].isin(suppliers)]
supplier_df['Supply'] = supplier_df['Origin'].apply(lambda x: suppliers[x].supply)
supplier_df

<ipython-input-46-ea14941771a6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supplier_df['Supply'] = supplier_df['Origin'].apply(lambda x: suppliers[x].supply)


,Origin,Flow Value,Supply
0,s1,3400.0,3400
1,s2,3600.0,3600
2,s3,2000.0,2000
3,s4,2500.0,2500
4,s5,1500.0,1500
5,s6,2000.0,2000
6,s7,5000.0,5000
